# Azure Computer Vision Dense Caption

### The Azure Computer Vision Dense Caption feature is capable of providing users with detailed object detection within an image's content. Subsequently, we can employ OpenAI to condense these results.

# Packages and Credentials

In [6]:
import requests
import json
from PIL import Image
from openai import cli
import matplotlib.pyplot as plt
import requests
import os
import azure.ai.vision as visionsdk
import openai
import logging

In [16]:

openai.api_type = 'azure'
openai.api_base = os.environ.get('openai.api_base')
openai.api_version = os.environ.get('openai.api_version')
openai.api_key = os.environ.get('openai.api_key')
vision_base= os.environ.get('cognitive_endpoint')
vision_key= os.environ.get('cognitive_key')

# Original Image

In [3]:
from IPython.display import Image
image_url = "https://www.realmenrealstyle.com/wp-content/uploads/2015/02/How-The-Car-You-Drive-Impacts-Your-Image.jpg"

# Display the image
Image(url=image_url)

In [4]:
def generate_caption(image_url):
    service_options = visionsdk.VisionServiceOptions(vision_base, vision_key)
    vision_source = visionsdk.VisionSource(url=image_url)
    image_analysis_options = visionsdk.ImageAnalysisOptions()
    image_analysis_options.features = (
        visionsdk.ImageAnalysisFeature.DENSE_CAPTIONS
    )
    image_analysis_options.language = 'en'
    image_analyzer = visionsdk.ImageAnalyzer(
        service_options, vision_source, image_analysis_options)
    result = image_analyzer.analyze()
    response = {}
    response_json={}
    if result.reason == visionsdk.ImageAnalysisResultReason.ANALYZED:
        if result.caption is not None:
            image_caption = result.caption.content
        else:
            image_caption = ''
        result_details = visionsdk.ImageAnalysisResultDetails.from_result(
            result)
        response_json = json.loads(result_details.json_result)
        response = openai.ChatCompletion.create(
            engine="gpt-35-turbo",
            messages=[
                {"role": "system", "content": "you are the AI assistant who extract main products of data"},
                {"role": "user", "content": f"can you give me what products are in this data? Just return the items having probability of more than 40 percent. {response_json} "}
            ])
    else:
        logging.info('Azure vision failed to generate dense caption of this image')
    formatted_result = response['choices'][0]['message']['content']
    return formatted_result, response_json

In [17]:
image_url = "https://www.realmenrealstyle.com/wp-content/uploads/2015/02/How-The-Car-You-Drive-Impacts-Your-Image.jpg"
text,caption=generate_caption(image_url)
print(text)

The following products are in the data with a probability of more than 40%:
- A man in a suit and tie talking on a cell phone next to a car
- A man in a white shirt and tie
- A person wearing black pants
- A person opening a car door
- A man in a suit and tie talking on a cell phone next to a car
- A close up of a car
